<a href="https://colab.research.google.com/github/matt-needle/project-data/blob/master/docs/examples/train_object_detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train an Object Detection Model with GeoAI

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/train_object_detection_model.ipynb)

## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [8]:
%pip install geoai-py

## Import libraries

In [9]:
import os
import geoai

out_folder = "output"

In [10]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


## Download sample data

## Train object detection model

In [11]:
import os

from google.colab import drive
drive.mount('/content/drive')

out_folder = "output"

print('images:', len(os.listdir('/content/drive/MyDrive/geoai/' + out_folder + '/images')), 'labels:', len(os.listdir('/content/drive/MyDrive/geoai/' + out_folder + '/labels')))

# Access your data
# !cp /content/drive/MyDrive/geoai/output .

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
images: 1087 labels: 1580


In [15]:
!cp /content/drive/MyDrive/geoai/output ./output -r

In [17]:
out_folder = '/content/output/'
# out_folder = '/content/drive/MyDrive/geoai/output'

geoai.train_MaskRCNN_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/models",
    num_channels=4,
    pretrained=True,
    batch_size=4,
    num_epochs=200,
    learning_rate=0.001,
    val_split=0.1,
)

Using device: cpu
Found 1580 image files and 1580 label files
Training on 1422 images, validating on 158 images
Epoch: 0, Batch: 0/356, Loss: 11.3177, Time: 86.05s
Epoch: 0, Batch: 10/356, Loss: 2.0823, Time: 785.95s
Epoch: 0, Batch: 20/356, Loss: 1.3965, Time: 796.52s


KeyboardInterrupt: 

In [ ]:
images_dir=f"{out_folder}/images"

os.listdir(images_dir)

## Run inference

In [ ]:
masks_path = "naip_test_prediction.tif"
model_path = f"{out_folder}/models/best_model.pth"

In [ ]:
geoai.object_detection(
    test_raster_path,
    masks_path,
    model_path,
    window_size=512,
    overlap=256,
    confidence_threshold=0.5,
    batch_size=4,
    num_channels=4,
)

## Vectorize masks

In [ ]:
output_path = "naip_test_prediction.geojson"
gdf = geoai.orthogonalize(masks_path, output_path, epsilon=2)

## Visualize results

In [ ]:
geoai.view_vector_interactive(output_path, tiles=test_raster_url)

In [ ]:
geoai.create_split_map(
    left_layer=output_path,
    right_layer=test_raster_url,
    left_args={"style": {"color": "red", "fillOpacity": 0.2}},
    basemap=test_raster_url,
)

![image](https://github.com/user-attachments/assets/8dfcc69e-7a6c-408a-9fae-10b81b7d85dc)